In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [2]:
"""
Restricted Master Problem (RMP)


"""
N = 3
M = 3
patterns = np.matrix([[2, 0, 0], [0, 1, 0], [0, 0, 1]])
print(patterns)
c = np.array([5, 5, 5])
rhs = np.array([30, 20, 40])
print(c)
model = gp.Model("cutting stock")
x = model.addVars(N, vtype=GRB.INTEGER, name="x")
cons = model.addConstrs((gp.quicksum(patterns[i, j] * x[j] for j in range(3)) >= rhs[i] for i in range(3)))
model.setObjective(gp.quicksum(c[i]*x[i] for i in range(3)))
model.write("master.lp")

[[2 0 0]
 [0 1 0]
 [0 0 1]]
[5 5 5]
Restricted license - for non-production use only - expires 2025-11-24


In [3]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x9d89c5da
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [5e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+01]
Found heuristic solution: objective 375.0000000
Presolve removed 3 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 375 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.750000000000e+02, best bound 3.750000000000e+02, gap 0.0000%


In [4]:
for v in model.getVars():
    print("%s = %g" % (v.VarName, v.x), end = " ")
print()
print("optimal: ", model.ObjVal)

x[0] = 15 x[1] = 20 x[2] = 40 
optimal:  375.0


In [5]:
model2 = gp.Model("cutting stock continous")
y = model2.addVars(N, vtype=GRB.CONTINUOUS, name="y")
cons = model2.addConstrs((gp.quicksum(patterns[i, j] * y[j] for j in range(3)) >= rhs[i] for i in range(3)))
model2.setObjective(gp.quicksum(c[i]*y[i] for i in range(3)))
model2.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x6021a812
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [5e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+01]
Presolve removed 3 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.750000000e+02


In [6]:
shadow = model2.getAttr('Pi', model2.getConstrs())
print("shadow: ", shadow)

shadow:  [2.5, 5.0, 5.0]


In [7]:
"""
Subproblem I

max 2.5 a1j + 5 a2j + 5 a3j - 5

s.t.
4 a1j + 5 a2j + 5 a3j <= 9
aij \in Z+
"""
reduced_cost = []
sub1 = gp.Model("subproblem 1")
demand = np.array([4, 5, 7])
a = sub1.addVars(3, vtype = GRB.INTEGER, name = "a")
sub1.addConstr((gp.quicksum(a[i] * demand[i] for i in range(3))) <= 9)
sub1.setObjective(gp.quicksum(shadow[i] * a[i] for i in range(3)) - 5, GRB.MAXIMIZE)
sub1.write("sub1.lp")
sub1.optimize()
for v in sub1.getVars():
    print("%s = %g" % (v.VarName, v.x), end = " ")
print()
print("optimal: ", sub1.ObjVal)
reduced_cost.append(sub1.ObjVal)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x2e342b45
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e+00, 7e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e+00, 9e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 2.5 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.500000000000e+00, best bound 2.500000000000e+00, gap 0.0000%
a[0] = 1 a[1] = 1 a[2] = -0 

In [8]:
"""
Subproblem II

max 2.5 a1j + 5 a2j + 5 a3j - 9

s.t.
4 a1j + 5 a2j + 5 a3j <= 14
aij \in Z+
"""
sub2 = gp.Model("subproblem 2")
demand = np.array([4, 5, 7])
a = sub2.addVars(3, vtype = GRB.INTEGER, name = "a")
sub2.addConstr((gp.quicksum(a[i] * demand[i] for i in range(3))) <= 14)
sub2.setObjective(gp.quicksum(shadow[i] * a[i] for i in range(3)) - 9, GRB.MAXIMIZE)
sub2.write("sub3.lp")
sub2.optimize()
for v in sub2.getVars():
    print("%s = %g" % (v.VarName, v.x), end = " ")
print()
print("optimal: ", sub2.ObjVal)
reduced_cost.append(sub2.ObjVal)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x7d90ec48
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e+00, 7e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective -1.5000000
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 3.5 -1.5 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.500000000000e+00, best bound 3.500000000000e+00, gap 0.0000%
a[0] = 1 a[1] = 2 a[2] = -

In [9]:
"""
Subproblem III

max 2.5 a1j + 5 a2j + 5 a3j - 10

s.t.
4 a1j + 5 a2j + 5 a3j <= 16
aij \in Z+
"""
sub3 = gp.Model("subproblem 3")
demand = np.array([4, 5, 7])
a = sub3.addVars(3, vtype = GRB.INTEGER, name = "a")
sub3.addConstr((gp.quicksum(a[i] * demand[i] for i in range(3))) <= 16)
sub3.setObjective(gp.quicksum(shadow[i] * a[i] for i in range(3)) - 10, GRB.MAXIMIZE)
sub3.write("sub3.lp")
sub3.optimize()
for v in sub3.getVars():
    print("%s = %g" % (v.VarName, v.x), end = " ")
print()
print("optimal: ", sub3.ObjVal)
reduced_cost.append(sub3.ObjVal)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x8fbdc27b
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e+00, 7e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 5 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%
a[0] = 0 a[1] = 3 a[2] = -0 
o

In [10]:
if any([ele < 0 for ele in reduced_cost]):
    print("Iteration")
else:
    print("Optimal found")

Optimal found
